## 공급량 데이터만 가지고 LSTM 모델 만들기 

In [2]:
import numpy as np
import pandas as pd 
import datetime
import matplotlib.pyplot as plt 
%matplotlib inline 

plt.rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False


df = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding = 'euc-kr')
test = pd.read_csv('가스공급량 수요예측모델개발_data/test.csv')
sub = pd.read_csv('가스공급량 수요예측모델개발_data/sample_submission.csv')

In [6]:
def tohour(val):
    if len(val) == 1:
        return '0'+val+':00:00'
    else:
        return val+':00:00'
    
df['시간_수정'] = df['시간'] -1 
df['시간_수정'] = df['시간_수정'].astype(str)
df['시간_수정'] = df['시간_수정'].apply(tohour)
df['연월일시간'] = df['연월일']+' '+df['시간_수정']
df['연월일시간'] = pd.to_datetime(df['연월일시간'])

A = df[df['구분'] =='A']
B = df[df['구분'] =='B']
C = df[df['구분'] =='C']
D = df[df['구분'] =='D']
E = df[df['구분'] =='E']
G = df[df['구분'] =='G']
H = df[df['구분'] =='H']

In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
# 학습용 데이터 세트 생성
# 'look_back=10'일 때 'x = [[[x0], .. [x10]] [[x1], .. [x11]] ..], y = [[x11], [x12] ..]'의 형태로 학습 데이터가 만들어진다.
def create_dataset(signal_data, look_back=1):  
    x_arr, y_arr = [], []
    for i in range(len(signal_data) - look_back):
        x_arr.append(signal_data[i:(i + look_back), 0])
        y_arr.append(signal_data[i + look_back, 0])

    x_arr = np.array(x_arr)
    x_arr = np.reshape(x_arr, (x_arr.shape[0], x_arr.shape[1], 1))
    return x_arr, np.array(y_arr)

### A 

In [9]:
scaler = MinMaxScaler(feature_range= (0,1))
공급량 = scaler.fit_transform(A['공급량'].to_frame())

In [10]:
train = 공급량[:int(len(공급량) * 0.5 )]
val = 공급량[int(len(공급량)*0.5): int(len(공급량)*0.75)]
test = 공급량[int(len(공급량)*0.75) : -1 ]

In [14]:
look_back = 90 * 24
batch_size = 100
x_train, y_train = create_dataset(train, look_back)  
x_val, y_val = create_dataset(val, look_back)  
x_test, y_test = create_dataset(test, look_back)  

In [15]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, Dropout
from tensorflow.keras.models import Model, Sequential

In [16]:
model = Sequential()  
model.add(LSTM(32, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))  
model.add(Dropout(0.2)) # overfitting을 막기 위해 20% 가량을 drop  
model.add(LSTM(32, batch_input_shape=(batch_size, look_back, 1), stateful=True))  
model.add(Dropout(0.2))  
model.add(Dense(1))

In [17]:
model.compile(loss='mae', optimizer='adam')

In [ ]:
model.fit(x_train, y_train, epochs = 100, batch_size = batch_size, validation_data = (x_val, y_val))

2021-11-12 17:54:58.940721: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-12 17:54:58.943155: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
